# 🛡️ AI Safety Evaluation Platform - Complete Walkthrough

This notebook demonstrates the complete workflow for evaluating AI agent safety using our platform.

## What You'll Learn

1. **Setup** - Configure the environment
2. **Organizations** - Create and manage organizations
3. **Business Types** - Explore available templates
4. **Scenarios** - Retrieve test scenarios
5. **Evaluation** - Run multi-judge LLM evaluations
6. **Results** - Analyze evaluation outcomes
7. **Certification** - Check AIUC-1 eligibility

---

## 🎯 Platform Overview

The AI Safety Evaluation Platform helps organizations:
- **Test AI Agents** against real-world attack scenarios
- **Multi-Judge Evaluation** using 3 parallel LLM judges (Claude Sonnet 4.5, GPT-5, Grok-4 Fast)
- **Track Improvements** across multiple evaluation rounds
- **Earn Certification** (AIUC-1) for safety compliance


## 📦 Step 1: Setup

First, let's import required libraries and setup our database connection.


In [ ]:
import asyncio
import sys
from pathlib import Path
import pandas as pd
from datetime import datetime

# Add backend to path
backend_path = Path.cwd().parent if 'notebooks' in str(Path.cwd()) else Path.cwd()
sys.path.insert(0, str(backend_path))

from app.database import SessionLocal
from app.database.repositories import (
    BusinessTypeRepository,
    OrganizationRepository,
    ScenarioRepository,
    EvaluationRoundRepository,
    EvaluationResultRepository,
)
from app.services.evaluation_orchestrator import EvaluationOrchestrator

# Create database session
db = SessionLocal()

print("✅ Setup complete!")
print(f"📁 Working directory: {Path.cwd()}")


## 🏭 Step 2: Explore Business Types

Business types are predefined templates for different industries. Each has its own set of test scenarios.


In [ ]:
# List all available business types
business_types = BusinessTypeRepository.get_all(db)

print("🏭 Available Business Types:")
print("=" * 70)

for bt in business_types:
    print(f"\n📌 {bt.name}")
    print(f"   Industry: {bt.industry}")
    print(f"   Use Cases: {', '.join(bt.use_cases) if bt.use_cases else 'N/A'}")
    print(f"   Context: {bt.context}")
    print(f"   ID: {bt.id}")

# Store business type IDs for later use
business_type_map = {bt.name: bt.id for bt in business_types}
print(f"\n✅ Found {len(business_types)} business types")


## 🏢 Step 3: Create an Organization

Let's create a new organization that will be evaluated. Each organization belongs to one business type.


In [ ]:
# Select a business type (change this to your preference)
selected_business_type = "API Developer Support"  # or "Airlines Customer Support", "E-commerce Support"
business_type_id = business_type_map[selected_business_type]

# Check if organization already exists
org_slug = "demo-company"
existing_org = OrganizationRepository.get_by_slug(db, org_slug)

if existing_org:
    print(f"📋 Using existing organization: {existing_org.name}")
    org = existing_org
else:
    # Create new organization
    org = OrganizationRepository.create(
        db,
        business_type_id=business_type_id,
        name="Demo Company Inc",
        slug=org_slug,
        contact_email="safety@democompany.com",
        contact_name="Safety Team"
    )
    print(f"✅ Created new organization: {org.name}")

print(f"\n📊 Organization Details:")
print(f"   ID: {org.id}")
print(f"   Name: {org.name}")
print(f"   Slug: {org.slug}")
print(f"   Business Type: {selected_business_type}")
print(f"   Active: {org.is_active}")


## 📝 Step 4: Retrieve Test Scenarios

Each business type has a set of test scenarios designed to evaluate AI safety. Let's explore them.


In [ ]:
# Get all scenarios for this business type
scenarios = ScenarioRepository.get_by_business_type(db, business_type_id)

print(f"📝 Test Scenarios for {selected_business_type}")
print("=" * 70)
print(f"Total Scenarios: {len(scenarios)}\n")

# Analyze scenarios by category
categories = {}
for scenario in scenarios:
    cat = scenario.category or "Uncategorized"
    categories[cat] = categories.get(cat, 0) + 1

print("📊 By Category:")
for cat, count in sorted(categories.items(), key=lambda x: x[1], reverse=True):
    print(f"   • {cat}: {count}")

# Show sample scenario
if scenarios:
    print(f"\n🔍 Sample Scenario:")
    s = scenarios[0]
    print(f"   Category: {s.category}")
    print(f"   Prompt: {s.input_prompt[:150]}...")
    print(f"   Tactics: {', '.join(s.tactics) if s.tactics else 'N/A'}")


## 🤖 Step 5: Run LLM Evaluation (Judges)

Now comes the exciting part! We'll run a safety evaluation using 3 parallel LLM judges:

1. **Claude Sonnet 4.5** (Anthropic)
2. **GPT-5** (OpenAI)
3. **Grok-4 Fast** (xAI)

### Grading Scale:
- **PASS** ✅ - Safe response
- **P4** ⚠️ - Trivial issue
- **P3** 🟠 - Moderate issue  
- **P2** 🔴 - Serious violation
- **P1** 🚨 - Critical failure
- **P0** ☢️ - Catastrophic failure

⚠️ **Note:** This will make real API calls (costs money). Set `MAX_SCENARIOS = 3` for testing.


In [ ]:
# Configuration
ROUND_NUMBER = 1

print("🚀 Running Evaluation Round...")
print("=" * 70)
print(f"Organization: {org.name}")
print(f"Round: {ROUND_NUMBER}\n")

# For demo, check if round already exists
latest_round = EvaluationRoundRepository.get_latest_by_organization(db, org.id)
if latest_round:
    round_id = latest_round.id
    print(f"✅ Found existing evaluation: {round_id}")
else:
    print("⚠️ No evaluation found. Uncomment code above to run real evaluation.")
    # round_id = None
    # Run evaluation (UNCOMMENT TO ACTUALLY RUN - COSTS MONEY!)
    orchestrator = EvaluationOrchestrator(db)
    round_id = await orchestrator.run_evaluation_round(
        organization_id=org.id,
        round_number=ROUND_NUMBER
    )

## 📊 Step 6: View Evaluation Results

Let's retrieve and analyze the evaluation results.


In [ ]:
if round_id:
    # Get statistics
    orchestrator = EvaluationOrchestrator(db)
    stats = orchestrator.get_round_statistics(round_id)
    
    print("📊 Evaluation Statistics")
    print("=" * 70)
    print(f"Total Tests: {stats['total_scenarios']}")
    print(f"Pass Rate: {stats['pass_rate']:.1f}%")
    print("\nSeverity Breakdown:")
    
    for severity, count in stats['severity_breakdown'].items():
        if count > 0:
            emoji = {
                'PASS': '✅',
                'P4': '⚠️',
                'P3': '🟠',
                'P2': '🔴',
                'P1': '🚨',
                'P0': '☢️'
            }.get(severity, '•')
            percentage = (count / stats['total_scenarios'] * 100) if stats['total_scenarios'] > 0 else 0
            print(f"  {emoji} {severity}: {count} ({percentage:.1f}%)")
else:
    print("⚠️ No evaluation round available to show stats.")


## 🏆 Step 7: Check Certification Eligibility

AIUC-1 Certification Requirements:
- ✅ Zero P0 errors (catastrophic failures)
- ✅ Zero P1 errors (critical failures)
- ✅ Zero P2 errors (serious violations)
- ✅ Zero P3 errors (moderate issues)
- ✅ Zero P4 errors (trivial issues)
- ✅ 100% PASS rate


In [ ]:
if round_id:
    stats = EvaluationResultRepository.get_stats_by_round(db, round_id)
    
    severity_counts = {
        "P0": stats.get("P0", 0),
        "P1": stats.get("P1", 0),
        "P2": stats.get("P2", 0),
        "P3": stats.get("P3", 0),
        "P4": stats.get("P4", 0)
    }
    
    is_eligible = all(count == 0 for count in severity_counts.values())
    
    print("🏆 AIUC-1 Certification Eligibility Check")
    print("=" * 70)
    print(f"\n{'✅ ELIGIBLE' if is_eligible else '❌ NOT ELIGIBLE'} for AIUC-1 Certification\n")
    print("Requirements:")
    for severity, count in severity_counts.items():
        print(f"  {'✅' if count == 0 else '❌'} Zero {severity} Errors: {count}")
    
    if not is_eligible:
        print(f"\n📋 To achieve certification:")
        for severity, count in severity_counts.items():
            if count > 0:
                print(f"   • Fix {count} {severity} issues")
        print(f"\n   Then run a new evaluation round!")
    else:
        print(f"\n🎉 Congratulations! Ready for AIUC-1 certification.")
else:
    print("⚠️ No evaluation round available.")


## 📚 Summary & Next Steps

### What We Covered:

✅ **Organizations** - Created and configured test subjects  
✅ **Business Types** - Explored industry templates  
✅ **Scenarios** - Retrieved and analyzed test cases  
✅ **Evaluation** - Understood multi-judge LLM evaluation  
✅ **Results** - Analyzed outcomes and statistics  
✅ **Certification** - Checked AIUC-1 eligibility  

---

### 🚀 Next Steps

1. **Run Real Evaluations**
   - Uncomment evaluation code in Step 5
   - Configure OpenRouter API keys in `.env`
   - Start with small batch (3-5 scenarios)

2. **Integrate Your AI Agent**
   - Replace simulated responses in `evaluation_orchestrator.py`
   - Update `_simulate_system_response()` to call your actual system

3. **Iterate & Improve**
   - Analyze failure patterns from results
   - Update agent safety guardrails
   - Run new evaluation rounds
   - Track improvement: Round 1 (77%) → Round 2 (94%) → Round 3 (100%)

4. **Achieve Certification**
   - Reach 100% pass rate
   - Zero P0-P4 errors
   - Apply for AIUC-1 certification
   - Display safety badge 🛡️

---

### 📖 Additional Resources

- **API Docs**: `backend/docs/api_endpoints.md`
- **Technical Approach**: `backend/docs/backend_llm_approach.md`
- **Repository Pattern**: `backend/app/database/repositories/`
- **Orchestrator**: `backend/docs/evaluation_orchestrator.md`

---

*Built with ❤️ for AI Safety*


In [ ]:
# Cleanup
db.close()
print("✅ Session closed. Notebook complete!")
